In [ ]:
import psycopg2
from nltk.corpus import stopwords
from collections import Counter
import matplotlib.pyplot as plt
import datetime
import csv
id=0
id2=0

#Create File for nodes
with open('nodes.csv', 'w', newline='') as csvfile:
        fieldnames = ['Id', 'Label','Timestamp','Size']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()


#Create File for edges
with open('edge.csv', 'w', newline='') as csvfile:
        fieldnames = ['Source', 'Target','Type','Kind','Id','Label','timeset','Weight']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()


try:
    conn = psycopg2.connect("dbname='****' user='******' host='*****' password='*******'")
except:
    print("I am unable to connect to the database")

result_time=[]
result_text=[]
result_text_1=[]
result_text_2=[]
result_text_3=[]
result_count=[]
result_count_1=[]
result_count_2=[]
result_count_3=[]
fin=[]
fintmp=[]
_fintmp=[]

cur = conn.cursor()

ye=['2014', '2015', '2016', '2017']
mo=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
#var=['Mensch | mensch | gesellschaft | Gesellschaft | NGO | ngo | Bürger | bürger | NPO | npo | gemein | Gemein | gemeinnützig | sozial | demokratie | demokrati']
var=['člověk | člověk | společnost | Společnost | nevládní | Nevládní | občan | Občan | Nezisková | nezisková | Charitativní | charitativní | sociální | Sociální | Sociálně | sociálně | demokracie | Demokracie | demokratický | Demokratický']

for y in ye:
    for m in mo:
    
        print(datetime.date(int(y),int(m),1))
        result_time.append(datetime.datetime(int(y),int(m),1))
        #Query the db
#        cur.execute("select document from standard where document @@ to_tsquery(%s) and to_char(pubdate,'yyyy')= %s and to_char(pubdate,'mm')= %s",(var[0],y,m))
        cur.execute("select document from novinky where document @@ to_tsquery(%s) and to_char(pubdate,'yyyy')= %s and to_char(pubdate,'mm')= %s",(var[0],y,m))

        rows = cur.fetchall()


        res=[]
        for row in rows:
            for s in row[0].split(':'):
                if len(s.split(' '))>1:
                    res.append(s.split(' ')[1])
                    
        dis=var[0].split(' | ')
            
        counts = Counter(res)

        bestof=counts.most_common(100)

        #s=set(stopwords.words('german'))
        s=set(stopwords.words('czech'))
        
        result_text_tmp=[]
        result_count_tmp=[]
        cnt=0
        for i in bestof:
            if i[0].replace("'",'') not in s and i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in dis:
                if cnt < 3:

                    if cnt == 0:
                        result_text_1.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                        result_count_1.append(i[1])
                    if cnt == 1:
                        result_text_2.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                        result_count_2.append(i[1])
                    if cnt == 2:
                        result_text_3.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                        result_count_3.append(i[1])
                        
                    print(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"), i[1])
                    result_text_tmp.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                    result_count_tmp.append(i[1])
                    if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in fin:
                        fin.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                    cnt+=1
        result_text.append(result_text_tmp)
        result_count.append(result_count_tmp)
        #id=0
        cnt=0
        
        
        
#write data into csv
 ###############################################################################
        with open('nodes.csv', 'a+', newline='') as csvfile:
                fieldnames = ['Id', 'Label','Timestamp','Size']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                #writer.writeheader()
                for i in bestof:
                    if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in s and i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in dis:
                        if cnt < 3:
                            #if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in _fintmp:
                            if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in _fintmp:
                                #_fintmp.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                                _fintmp.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                                #fintmp.append([i[0].replace("'",'').encode('utf8').decode("ascii","ignore"), id])
                                fintmp.append([i[0].replace("'",'').encode('utf8').decode("ascii","ignore"), id])
                                #print(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"), i[1])
                                #writer.writerow({'Id': str(id), 'Label': i[0].replace("'",'').encode('utf8').decode("ascii","ignore"), 'Timestamp':datetime.date(int(y),int(m),1), 'Size':float(i[1])})
                                writer.writerow({'Id': str(id), 'Label': i[0].replace("'",'').encode('utf8').decode("ascii","ignore"), 'Timestamp':datetime.date(int(y),int(m),1), 'Size':float(i[1])})
                                id+=1
                            cnt+=1
############################################################################### 
        #id=0
        cnt=0
        acttop3=[]
###############################################################################
        with open('edge.csv', 'a+', newline='') as csvfile:
                fieldnames = ['Source', 'Target','Type','Kind','Id','Label','Timeset','Weight']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                #writer.writeheader()
                for i in bestof:
                    if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in s and i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in dis:
                        if cnt < 3:
                            if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in acttop3:
                                acttop3.append(i[0].replace("'",'').encode('utf8').decode("ascii","ignore"))
                                cnt+=1
                cnt=0
                print(acttop3)
                for i in bestof:
                    if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in s and i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in dis:
                        if cnt < 3:
                            for node in fintmp:
                                #for top3  in acttop3:
                                #print('-----------')
                                #print(cnt)
                                #print(top3)
                                if i[0].replace("'",'').encode('utf8').decode("ascii","ignore") not in s and i[0].replace("'",'').encode('utf8').decode("ascii","ignore") == node[0]:
                                    for inside in acttop3:
                                        if inside != i[0].replace("'",'').encode('utf8').decode("ascii","ignore"):
                                            for z in fintmp:
                                                if inside == z[0]:
                                                    #writer.writerow({'Source':node[1], 'Target':'','Label': i[0].replace("'",'').encode('utf8').decode("ascii","ignore"),'Type':'Undirected' 'Timestamp':datetime.date(int(y),int(m),1), 'Size':float(i[1])})
                                                    writer.writerow({'Source':node[1], 'Target':z[1],'Type':'Undirected','Kind':'','Id':id2,'Label':i[0].replace("'",'').encode('utf8').decode("ascii","ignore"),'Timeset':'', 'Weight':float(i[1])})
                                                    id2+=1
                        cnt+=1
############################################################################### 
conn.close()